In [1]:
import torch;import torch.nn as nn;import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from layer.modules.ssd import SSD
from layer.function.iou import match
a = torch.rand(2,3,300,300)
b = SSD()
c = b(a)
d = torch.tensor([[0.0,0.0,1.0,1.0,2.0],[0.5,0.5,1.0,1.0,5.0]]).reshape(2,-1) #fake truthbox
d1 = torch.tensor((0.0,0.0,1.0,1.0,2.0)).reshape(1,-1)

In [2]:
matchbox = torch.cuda.FloatTensor(*c[1].shape)
loc_conf = torch.cuda.FloatTensor(*c[0].shape)

for i,j in enumerate((d,d1)):
    e = match(c[2],j)
    matchbox[i].zero_().scatter_(1,e[1].unsqueeze(1),1.0)
    loc_conf[i] = e[0]

In [4]:
matchbox.size()

torch.Size([2, 8732, 21])

In [27]:
g = matchbox[:,:,:-1].long().sum(1,keepdim=True)

In [28]:
num_neg = torch.clamp(g.sum(1,keepdim=True),max=matchbox.size(1)-1)

In [7]:
loc = c[0]

In [8]:
conf = c[1]

In [9]:
loss_c = conf.clone()

In [11]:
matchbox = matchbox.byte()

In [12]:
pos = matchbox[:,:,:-1].any(2)

In [15]:
loss_c[pos] = 0

In [19]:
_, loss_idx = loss_c.sort(1,descending=True)
_, idx_rank = loss_idx.sort(1)

In [33]:
neg = idx_rank[:,:,:-1] < num_neg.expand_as(idx_rank[:,:,:-1])